In [1]:
#Libs
import pandas as pd
import json
import re
from google.colab import files
import io

In [2]:
print("Files Please!")
uploaded = files.upload()

Files Please!


Saving orders.csv to orders.csv
Saving restaurants.sql to restaurants.sql
Saving users.json to users.json


In [3]:
# 1. Load Orders
df_orders = pd.read_csv(io.BytesIO(uploaded['orders.csv']))

# 2. Load Users
df_users = pd.read_json(io.BytesIO(uploaded['users.json']))

# 3. Parse SQL
restaurant_data = []
sql_content = uploaded['restaurants.sql'].decode('utf-8').splitlines()

for line in sql_content:
    if line.strip().startswith("INSERT INTO"):
        # Extract values inside parenthesis
        match = re.search(r"VALUES\s*\((.*?)\);", line)
        if match:
            # Clean and split data
            parts = [p.strip().strip("'") for p in match.group(1).split(',')]
            restaurant_data.append({
                'restaurant_id': int(parts[0]),
                'restaurant_name_master': parts[1],
                'cuisine': parts[2],
                'rating': float(parts[3])
            })

df_restaurants = pd.DataFrame(restaurant_data)

# 4. Merge Data (Left Joins)
# Merge Orders + Users
df_merged = pd.merge(df_orders, df_users, on='user_id', how='left')

# Merge result + Restaurants
df_final = pd.merge(df_merged, df_restaurants, on='restaurant_id', how='left')

# 5. Clean Data types
df_final['order_date'] = pd.to_datetime(df_final['order_date'], dayfirst=True)

# 6. Save Final Dataset
df_final.to_csv('final_food_delivery_dataset.csv', index=False)
print("ETL Complete. 'final_food_delivery_dataset.csv' created.")
print("Total Rows:", len(df_final))
print(df_final.head())

ETL Complete. 'final_food_delivery_dataset.csv' created.
Total Rows: 10000
   order_id  user_id  restaurant_id order_date  total_amount  \
0         1     2508            450 2023-02-18        842.97   
1         2     2693            309 2023-01-18        546.68   
2         3     2084            107 2023-07-15        163.93   
3         4      319            224 2023-10-04       1155.97   
4         5     1064            293 2023-12-25       1321.91   

                  restaurant_name       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_master  cuisine  rating  
0         Restaurant_450  Mexican     3.2  
1         Restaurant_309   Indi

In [8]:
# Load dataset
df = pd.read_csv('final_food_delivery_dataset.csv')
df['order_date'] = pd.to_datetime(df['order_date'])

# Q1: Highest Revenue City (Gold)
gold_rev = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum()
print(f"1. Highest Revenue City (Gold): {gold_rev.idxmax()} ({gold_rev.max()})")

# Q2: Highest AOV Cuisine
cuisine_aov = df.groupby('cuisine')['total_amount'].mean()
print(f"2. Highest AOV Cuisine: {cuisine_aov.idxmax()} ({cuisine_aov.max()})")

# Q3: Users spend > 1000
user_spend = df.groupby('user_id')['total_amount'].sum()
print(f"3. Users spend > 1000: {user_spend[user_spend > 1000].count()}")

# Q4: Rating Range Revenue
# Create bins: 3.0-3.5, 3.6-4.0, 4.1-4.5, 4.6-5.0
bins = [2.9, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']
df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels)
print(f"4. Highest Rev Rating Range: {df.groupby('rating_range', observed=False)['total_amount'].sum().idxmax()}")

# Q6: Highest Revenue Quarter
df['quarter'] = df['order_date'].dt.to_period('Q')
print(f"6. Highest Revenue Quarter: {df.groupby('quarter')['total_amount'].sum().idxmax()}")

# Q7: Gold AOV City
gold_aov_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean()
print(f"7. Highest Gold AOV City: {gold_aov_city.idxmax()}")

# Q8: Lowest Restaurant Count Cuisine
print(f"8. Lowest Rest. Count Cuisine: {df.groupby('cuisine')['restaurant_id'].nunique().idxmin()}")

# Q9: Gold Order Percentage
gold_pct = (len(df[df['membership'] == 'Gold']) / len(df)) * 100
print(f"9. Gold Order %: {round(gold_pct)}%")

# Q10 Check: Specific Restaurant AOV (< 20 orders)
target_rests = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
stats = df.groupby('restaurant_name')['total_amount'].agg(['mean', 'count'])
filtered = stats[stats.index.isin(target_rests) & (stats['count'] < 20)]
print("\n--- Restaurant AOV (<20 orders) ---")
print(filtered.sort_values('mean', ascending=False))

1. Highest Revenue City (Gold): Chennai (1080909.79)
2. Highest AOV Cuisine: Mexican (808.0213444401395)
3. Users spend > 1000: 2544
4. Highest Rev Rating Range: 4.6-5.0
6. Highest Revenue Quarter: 2023Q3
7. Highest Gold AOV City: Chennai
8. Lowest Rest. Count Cuisine: Chinese
9. Gold Order %: 50%

--- Restaurant AOV (<20 orders) ---
                           mean  count
restaurant_name                       
Ruchi Foods Chinese  686.603158     19


In [9]:
# Numerical Q1: Total Gold Orders
print(f"Gold Orders: {len(df[df['membership'] == 'Gold'])}")

# Numerical Q2: Hyderabad Revenue
print(f"Hyderabad Rev: {round(df[df['city'] == 'Hyderabad']['total_amount'].sum())}")

# Numerical Q3: Distinct Users
print(f"Distinct Users: {df['user_id'].nunique()}")

# Numerical Q4: Gold AOV
print(f"Gold AOV: {round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)}")

# Numerical Q5: Orders Rating >= 4.5
print(f"Orders Rating >= 4.5: {len(df[df['rating'] >= 4.5])}")

# Numerical Q6: Orders in Top Gold Revenue City
gold_df = df[df['membership'] == 'Gold']
top_city = gold_df.groupby('city')['total_amount'].sum().idxmax()
print(f"Orders in {top_city} (Gold only): {len(gold_df[gold_df['city'] == top_city])}")

Gold Orders: 4987
Hyderabad Rev: 1889367
Distinct Users: 2883
Gold AOV: 797.15
Orders Rating >= 4.5: 3374
Orders in Chennai (Gold only): 1337


In [12]:
print(f"1. Total Rows: {len(df)}")

user_counts = df['user_id'].value_counts()
print(f"2. User details appear: multiple times (Max duplication: {user_counts.max()})")

1. Total Rows: 10000
2. User details appear: multiple times (Max duplication: 13)
